In [8]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [9]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add

# ResNet-34 from Scratch
## General structure 34-layer residual
<img src="./pictures/structure.png" width=200>

### The matter of skip connections
- In early CNN architectures as more and more layers were added to the Neural Network it was observed that the performance of the model started dropping
- The vanishing gradient problem caused this problem
- The solution for that is to use a skip connection 

## Identity Block 
### Structure
<img src="./pictures/identity_block.png" width=200>

### General
- The Identity Block directly adds the residue to the output
- Padding is the same for every layer, because we have to maintain the shape of our input until we add it to the residue
- The Identity Block is used 

### Code

In [10]:
def identity_block(x, filter):
    x_skip = x

    x = Conv2D(filter, (3,3), padding='same')(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)

    x = Conv2D(filter, (3,3), padding='same')(x)
    x = BatchNormalization(axis=3)(x)

    # Add residue
    x = Add()([x, x_skip])
    x = Activation('relu')(x)

    return x

## Convolutional Block
### Structure

<img src="./pictures/convolutional_block.png" width=200>

### General
- The Convolutional block performs a convolution followed by Batch Normalisation on the residue before adding it to the output

### Code

In [11]:
def convolutional_block(x, filter):
    x_skip = x

    x = Conv2D(filter, (3,3), padding='same', strides=(2,2))(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)

    x = Conv2D(filter, (3,3), padding='same')(x)
    x = BatchNormalization(axis=3)(x)

    # Processing Residue with conv(1,1)
    x_skip = Conv2D(filter, (1,1), strides=(2,2))(x_skip)

    x = Add()([x, x_skip])
    x = Activation('relu')(x)

    return x

## ResNet-34 Model
### Structure

<img src="./pictures/resnet_34_structure.png" width=500>

### Code

In [12]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, AveragePooling2D, MaxPool2D, ZeroPadding2D, Input

In [13]:
def ResNet34(shape = (32, 32, 3), classes=10):
    # Input Layer
    x_input = Input(shape)
    x = ZeroPadding2D((3, 3))(x_input)

    # Inital convolutional layer with maxPool
    x = Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)

    # Sub-blocks and inital filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64

    # ResNet blocks
    for i in range(4):
        if i == 0:
            for _ in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            filter_size = filter_size * 2
            x = convolutional_block(x, filter_size)
            for _ in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    
    # End Dense Network
    x = AveragePooling2D((2, 2), padding='same')(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(classes, activation='softmax')(x)

    model = Model(inputs=x_input, outputs=x, name="ResNet34 by Dennis")
    return model

In [14]:
test = ResNet34()
test.summary()

Model: "ResNet34 by Dennis"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 38, 38, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 19, 19, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
batch_normalization_9 (BatchNor (None, 19, 19, 64)   256         conv2d_10[0][0]                  
_________________________________________________________________________________